In [1]:
import requests
import json
import pandas as pd

In [2]:
headers = {
    "Host": "api.sofascore.com",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:109.0) Gecko/20100101 Firefox/118.0",
    "Accept": "*/*",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate",
    "Referer": "https://www.sofascore.com/",
    "Origin": "https://www.sofascore.com",
    "DNT": "1",
    "Connection": "keep-alive",
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "same-site",
    "Pragma": "no-cache",
    "Cache-Control": "no-cache",
    "TE": "trailers",
}

In [3]:
def get_stats_match(id):
    url_stats_base = f'https://api.sofascore.com/api/v1/event/{id}/statistics'
    response = requests.get(url_stats_base, headers=headers)
    data_raw = response.json()['statistics']
    xg_raw = data_raw[1]['groups'][0]['statisticsItems'][0]
    xg_home = xg_raw['homeValue']
    xg_away = xg_raw['awayValue']

    return xg_home, xg_away
    

In [4]:
def matches_id(round):
    url_rounds = f'https://api.sofascore.com/api/v1/unique-tournament/325/season/48982/events/round/{round}'
    response = requests.get(url_rounds, headers=headers)
    data_raw = response.json()['events']
    round_data = []
    for game in data_raw:
        try:
            round_data.append(
            {
                'round': round,
                'game_id': game['id'], 
                'home': game['homeTeam']['shortName'],
                'home_score': game['homeScore']['current'], 
                'away': game['awayTeam']['shortName'],
                'away_score': game['awayScore']['current'],
                }
            )
        except:
            pass
    df = pd.DataFrame(round_data)
    # Apply the function to each row in the DataFrame
    df['xg_home'], df['xg_away'] = zip(*df['game_id'].apply(lambda x: get_stats_match(x)))
    df['xg_result'] = df['xg_home'] - df['xg_away']

    return df

In [5]:
times_serie_a = [
    "Athletico", "América-MG", "Atlético-MG", "Bahia", "Cuiabá", "Goiás",
    "Corinthians", "Flamengo", "Fluminense", "Grêmio", "Internacional",
    "Botafogo", "Palmeiras", "Santos", "São Paulo", "Cruzeiro",
    "Fortaleza", "Coritiba", "RB Bragantino", "Vasco"
]
len(times_serie_a)

20

In [6]:
tabela = pd.DataFrame(
    data = {
        'time': times_serie_a,
        'vitorias': len(times_serie_a)*[0],
        'empates': len(times_serie_a)*[0],
        'derrotas': len(times_serie_a)*[0],
        'gols_pro': len(times_serie_a)*[0.0],
        'gols_contra': len(times_serie_a)*[0.0],
        }
)
tabela

,time,vitorias,empates,derrotas,gols_pro,gols_contra
0,Athletico,0,0,0,0.0,0.0
1,América-MG,0,0,0,0.0,0.0
2,Atlético-MG,0,0,0,0.0,0.0
3,Bahia,0,0,0,0.0,0.0
4,Cuiabá,0,0,0,0.0,0.0
5,Goiás,0,0,0,0.0,0.0
6,Corinthians,0,0,0,0.0,0.0
7,Flamengo,0,0,0,0.0,0.0
8,Fluminense,0,0,0,0.0,0.0
9,Grêmio,0,0,0,0.0,0.0


In [7]:
def count_round(round, tabela):
    for index, row in round.iterrows():
        if abs(row['xg_result']) > 0.1:
            if row['xg_result'] > 0:
                tabela.loc[tabela['time'] == row['home'], 'vitorias'] += 1
                tabela.loc[tabela['time'] == row['away'], 'derrotas'] += 1
            else:
                tabela.loc[tabela['time'] == row['away'], 'vitorias'] += 1
                tabela.loc[tabela['time'] == row['home'], 'derrotas'] += 1
        else:
            tabela.loc[tabela['time'] == row['home'], 'empates'] += 1
            tabela.loc[tabela['time'] == row['away'], 'empates'] += 1
        tabela.loc[tabela['time'] == row['home'], 'gols_pro'] += row['xg_home']
        tabela.loc[tabela['time'] == row['home'], 'gols_contra'] += row['xg_away']
        tabela.loc[tabela['time'] == row['away'], 'gols_pro'] += row['xg_away']
        tabela.loc[tabela['time'] == row['away'], 'gols_contra'] += row['xg_home']

    return tabela

In [8]:
for round in range(1, 27):
    print(f'Atualizando para rodada {round}...')
    df = matches_id(round)
    count_round(df, tabela)

Atualizando para rodada 1...
Atualizando para rodada 2...
Atualizando para rodada 3...
Atualizando para rodada 4...
Atualizando para rodada 5...
Atualizando para rodada 6...
Atualizando para rodada 7...
Atualizando para rodada 8...
Atualizando para rodada 9...
Atualizando para rodada 10...
Atualizando para rodada 11...
Atualizando para rodada 12...
Atualizando para rodada 13...
Atualizando para rodada 14...
Atualizando para rodada 15...
Atualizando para rodada 16...
Atualizando para rodada 17...
Atualizando para rodada 18...
Atualizando para rodada 19...
Atualizando para rodada 20...
Atualizando para rodada 21...
Atualizando para rodada 22...
Atualizando para rodada 23...
Atualizando para rodada 24...
Atualizando para rodada 25...
Atualizando para rodada 26...


In [9]:
tabela['saldo_xg'] = tabela['gols_pro'] - tabela['gols_contra']
tabela['pontos'] = 3*tabela['vitorias'] + 1*tabela['empates']

In [10]:
tabela.sort_values(['pontos', 'vitorias', 'saldo_xg', 'gols_pro'], ascending=False, ignore_index=True)

,time,vitorias,empates,derrotas,gols_pro,gols_contra,saldo_xg,pontos
0,São Paulo,16,2,8,19.27,10.91,8.36,50
1,Palmeiras,14,4,8,16.77,13.42,3.35,46
2,Fortaleza,14,3,9,17.76,13.60,4.16,45
3,Cuiabá,14,3,8,11.31,9.42,1.89,45
4,Botafogo,14,3,9,13.88,12.13,1.75,45
5,Flamengo,13,4,9,16.61,13.38,3.23,43
6,América-MG,13,4,9,17.06,15.89,1.17,43
7,RB Bragantino,13,3,10,16.62,14.81,1.81,42
8,Cruzeiro,12,6,7,13.51,11.12,2.39,42
9,Fluminense,11,4,11,16.09,15.31,0.78,37


In [11]:
get_stats_match(11067539)

KeyError: 'statistics'